### ДЗ 1

Мы -- продуктовый магазин. Необходимо хранить ассортимент и продажи.

In [1]:
import sqlite3
import os
import random

In [2]:
with open('assortment.txt', 'r', encoding='utf-8') as f:
    assortment_arr = f.read().replace('"', '').split('\n')

In [3]:
sales_arr = [random.randint(0,100000) for _ in range(len(assortment_arr))]

**Создаем базу данных**

In [4]:
conn = sqlite3.connect('store.db')
cur = conn.cursor()

In [5]:
cur.execute('CREATE TABLE if not exists grocery (assortment text, sales int)')

In [6]:
for product, quantity in zip(assortment_arr, sales_arr):
    cur.execute('INSERT INTO grocery VALUES(?, ?)', [product, quantity])

conn.commit()

### Добавление данных, введенных пользователем:

In [7]:
def input_data():
    user_input = input('Enter the information in the following format: ProductName, SalesQuantity. Then press Enter.\n')
    
    if ''.join(user_input.split()) == '':
        user_input = input('It seems like you have entered an empty search query. Please, try again:\n')
    
    try:   
        user_input = user_input.split(',')
        product_name = user_input[0]
        sales_quantity = int(user_input[1].strip(' '))

    except IndexError as e:
        user_input = input('Please, check your query!' + \
                           '\nEnter the information in the following format: ProductName, SalesQuantity.\n')
        user_input = user_input.split(',')
        product_name = user_input[0]
        sales_quantity = int(user_input[1].strip(' '))
    
    return product_name, sales_quantity

In [8]:
def add_values(product_name, sales_quantity):
    cur.execute('SELECT * FROM grocery')
    rows = cur.fetchall()
    check_rows = []
    for i in range(len(rows)):
        check_rows.append(rows[i][0].lower())

    rows_dict = dict(rows)
    
    if product_name.lower() in check_rows:
        print('\nThis product already exists in "store.db"\nSalesQuantity will be added to the current value.')
        quantity = rows_dict[product_name.title()]
        new_value = int(quantity + sales_quantity)
        for key, value in rows_dict.items():
            rows_dict[product_name.title()] = new_value
        cur.execute('UPDATE grocery SET sales = (?) WHERE assortment = (?)', [new_value, product_name.title()])
        print('Previous values: %s, %d' % (product_name.title(), quantity))
        print('Updated values: %s, %d (+ %d PC)' % (product_name.title(), new_value, sales_quantity))
        
    else:
        cur.execute('INSERT INTO grocery VALUES (?, ?)', [product_name.title(), sales_quantity])
        print('\nNew record: "%s, %d". Added to "store.db"' % (product_name.title(), sales_quantity))
    
    conn.commit()

### Поиск по базе по запросу пользователя

In [9]:
def search_input():
    user_input = input('Enter a product name or quantity to search. Then press Enter.\n')
    
    if ''.join(user_input.split()) == '':
        user_input = input('It seems like you have entered an empty search query. Please, try again:\n')
    
    return user_input

In [10]:
def num_there(s):
    return any(i.isdigit() for i in s)

In [11]:
def search_values(user_input):
    cur.execute('SELECT * FROM grocery')
    rows = cur.fetchall()
    check_rows = []
    for i in range(len(rows)):
        check_rows.append(rows[i][0].lower())

    rows_dict = dict(rows)
    
    if not num_there(user_input):
        if user_input.lower() in check_rows:
            cur.execute('SELECT * FROM grocery WHERE assortment=?', [user_input.title()])
            result = cur.fetchall()
            for res in result:
                print('\nSearch result for your query:\nProduct name: %s\nQuantity: %d' % (res[0], res[1]))
                
        else:
            new_input = input('It seems like there is nothing in "store.db" that suits your request.' + \
                           '\nIf you want to try another request, please, enter a new one.' + \
                           '\nIf you want to add the query that you have entered into "store.db", then enter add:\n')
            
            if new_input.lower() in check_rows:
                cur.execute('SELECT * FROM grocery WHERE assortment=?', [new_input.title()])
                result = cur.fetchall()
                for res in result:
                    print('\nSearch result for your query:\nProduct name: %s\nQuantity: %d' % (res[0], res[1]))
            
            if new_input == 'add':
                product_name = user_input
                sales_quantity = int(input('Please, enter the quantity value:\n'))
                add_values(product_name, sales_quantity)
                
    elif num_there(user_input):
        cur.execute('SELECT * FROM grocery WHERE sales=?', [int(user_input)])
        result = cur.fetchall()
        
        if result == []:
            new_input = input('It seems like there is nothing in "store.db" that suits your request.' + \
                           '\nIf you want to try another request, please, enter a new one.' + \
                           '\nIf you want to add the query that you have entered into "store.db", then enter add:\n')
            
            if new_input.lower() in check_rows:
                cur.execute('SELECT * FROM grocery WHERE assortment=?', [new_input.title()])
                result = cur.fetchall()
                for res in result:
                    print('\nSearch result for your query:\nProduct name: %s\nQuantity: %d' % (res[0], res[1]))
            
            if new_input == 'add':
                sales_quantity = int(user_input)
                product_name = input('Please, enter product name:\n')
                add_values(product_name, sales_quantity)
        
        else:
            for res in result:
                print('\nSearch result for your query:\nProduct name: %s\nQuantity: %d' % (res[0], res[1]))
        
    conn.commit()

In [12]:
def main():
    while True:
        db_options = input('If you want to add new values to "store.db", enter "add",\n' +
                           'If you want to find a value in "store.db", enter "search":\n')
        
        if db_options == 'add':
            product_name, sales_quantity = input_data()
            add_values(product_name, sales_quantity)
            break

        elif db_options == 'search':
            user_input = search_input()
            search_values(user_input)
            break

#### Примеры

**Добавление нового элемента**

In [13]:
main()

If you want to add new values to "store.db", enter "add",
If you want to find a value in "store.db", enter "search":
add
Enter the information in the following format: ProductName, SalesQuantity. Then press Enter.
молоко, 150

New record: "Молоко, 150". Added to "store.db"


**Добавление существующего элемента**

In [14]:
main()

If you want to add new values to "store.db", enter "add",
If you want to find a value in "store.db", enter "search":
add
Enter the information in the following format: ProductName, SalesQuantity. Then press Enter.
твикс, 351

This product already exists in "store.db"
SalesQuantity will be added to the current value.
Previous values: Твикс, 50067
Updated values: Твикс, 50418 (+ 351 PC)


**Поиск элемента**

In [15]:
main()

If you want to add new values to "store.db", enter "add",
If you want to find a value in "store.db", enter "search":
search
Enter a product name or quantity to search. Then press Enter.
рис

Search result for your query:
Product name: Рис
Quantity: 16746


**Поиск по колонке sales (кол-во продаж)**

In [18]:
main()

If you want to add new values to "store.db", enter "add",
If you want to find a value in "store.db", enter "search":
search
Enter a product name or quantity to search. Then press Enter.
21567

Search result for your query:
Product name: Вафли
Quantity: 21567


**Поиск несуществующего элемента**

In [19]:
main()

If you want to add new values to "store.db", enter "add",
If you want to find a value in "store.db", enter "search":
search
Enter a product name or quantity to search. Then press Enter.
пирожное
It seems like there is nothing in "store.db" that suits your request.
If you want to try another request, please, enter a new one.
If you want to add the query that you have entered into "store.db", then enter add:
add
Please, enter the quantity value:
567

New record: "Пирожное, 567". Added to "store.db"
